In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torchvision import datasets
from torchvision.transforms import transforms

In [6]:
# 연속해서 이미지를 전처리해주는 함수 Compose

data_transform = transforms.Compose(
    [
        transforms.ToTensor(),
    ]
)

In [7]:
train_data = datasets.MNIST(root='./',train=True,download=True,transform=data_transform)

In [8]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
           )

In [9]:
train_data.data

tensor([[[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        ...,

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0,

In [10]:
train_data.data.shape

torch.Size([60000, 28, 28])

In [20]:
data_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Resize(32),
        transforms.Normalize((0.5),(1.0))   # 평균 0.5 = 전체 이미지의 색상값을 다더해서 평균내서 0.5 . 간이정규화
    ]
)
train_data = datasets.MNIST(root='./',train=True,download=True,transform=data_transform)
test_data = datasets.MNIST(root='./', train=False, download=True,transform=data_transform)

In [21]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: ./
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
               Resize(size=32, interpolation=bilinear, max_size=None, antialias=True)
               Normalize(mean=0.5, std=1.0)
           )

In [22]:
test_data.data

tensor([[[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        ...,

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0,

In [23]:
test_data.data.shape

torch.Size([10000, 28, 28])

In [24]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_data,batch_size=32,shuffle=True)
test_loader = DataLoader(test_data,batch_size=32)

In [25]:
next(iter(train_loader))[0].shape

torch.Size([32, 1, 32, 32])

In [27]:
class Lenet(nn.Module):
    def __init__(self):
        super(Lenet,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels= 6, kernel_size=5, stride=1)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1)
        self.fc1 = nn.Linear(in_features=120, out_features=84)
        self.fc2 = nn.Linear(in_features=84, out_features=10)

    def forward(self,x):
        x = self.conv1(x)   # 신경망 통과
        x = F.tanh(x) # 활성화함수통과할차례
        x = F.max_pool2d(x,2,2)
        x = self.conv2(x)
        x = F.max_pool2d(x,2,2)
        x = F.tanh(x)
        x = self.conv3(x)
        x = F.tanh(x)
        x = x.view(-1,120)
        x = self.fc1(x)     # F6 통과 84개로 통과
        x = F.tanh(x)   # 활성화함수
        x = self.fc2(x)
        x = F.tanh(x)
        return x

model = Lenet()
model
    

Lenet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=120, out_features=84, bias=True)
  (fc2): Linear(in_features=84, out_features=10, bias=True)
)

In [ ]:
list(model.parameters())    # 아직 학습 안 된 모델

[Parameter containing:
 tensor([[[[ 0.1413, -0.1206,  0.1022,  0.0901,  0.0803],
           [-0.0803,  0.0416, -0.0573,  0.1100,  0.0175],
           [-0.0403,  0.1664, -0.0512, -0.1114,  0.0388],
           [-0.0847,  0.0147,  0.1779,  0.0376, -0.1909],
           [-0.1202, -0.1650, -0.0822, -0.0471, -0.0868]]],
 
 
         [[[ 0.1213, -0.1111, -0.0119,  0.0635,  0.0128],
           [ 0.0705,  0.0847,  0.1140,  0.1843,  0.1096],
           [-0.1910,  0.1488, -0.1097,  0.1417,  0.1720],
           [ 0.1407,  0.1928,  0.1518, -0.0591, -0.1278],
           [-0.1245, -0.0392, -0.1690, -0.1009,  0.0275]]],
 
 
         [[[-0.1961,  0.0055, -0.1363,  0.0265,  0.0786],
           [ 0.1199, -0.0927, -0.1955, -0.0204, -0.1351],
           [-0.1313,  0.1521,  0.0871, -0.0735, -0.0739],
           [ 0.0349,  0.0296,  0.1073,  0.0201,  0.1976],
           [ 0.1922,  0.0493, -0.0486, -0.0982, -0.1635]]],
 
 
         [[[ 0.0682,  0.0122, -0.1641,  0.1470, -0.0241],
           [-0.1336,  0.0752,  

In [29]:
model(torch.randn((1,1,32,32)))

tensor([[ 0.1021,  0.0391,  0.0566,  0.0888,  0.0117,  0.0763,  0.0385, -0.1142,
          0.0442,  0.0517]], grad_fn=<TanhBackward0>)

In [33]:
# torchsummary라는 라이브러리는 이미지 사이즈가 어떻게되는지 다 체크가 된다

from torchsummary import summary

summary(model, input_size=(1,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             156
            Conv2d-2           [-1, 16, 10, 10]           2,416
            Conv2d-3            [-1, 120, 1, 1]          48,120
            Linear-4                   [-1, 84]          10,164
            Linear-5                   [-1, 10]             850
Total params: 61,706
Trainable params: 61,706
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.05
Params size (MB): 0.24
Estimated Total Size (MB): 0.29
----------------------------------------------------------------


In [32]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

Lenet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=120, out_features=84, bias=True)
  (fc2): Linear(in_features=84, out_features=10, bias=True)
)

In [34]:
device

'cuda'

In [36]:
#학습할준비
import tqdm   # 라이브러리 진행률 확인
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

optim = Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
epoch = 10

count = 0

for epoch in range(10):
    train_tqdm = tqdm.tqdm(train_loader)
    for data,label in train_tqdm:
        data = data.to(device)
        label = label.to(device)
        optim.zero_grad()
        pred = model(data)
        loss = criterion(pred,label)
        writer.add_scalar("Loss.train", loss, count)    # tensorboard에 그래프로 설정할 예정
        count += 1
        loss.backward()
        optim.step()

        train_tqdm.set_description(f'epoch : {epoch+1} loss: {loss.item()}')      # 몇번째 epoch인지, 손실값은  어떤지


epoch : 10 loss: 0.7993489503860474: 100%|██████████| 1875/1875 [00:35<00:00, 52.34it/s]


In [ ]:
#runs 폴더 만들어지면서 기록돼있다
# 터미널에서 아래 실행
# (jwdeep) C:\jwdeep>tensorboard --logdir=runs  


In [37]:
model.eval()   # 모델을 테스트용으로 사용하겠다

Lenet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=120, out_features=84, bias=True)
  (fc2): Linear(in_features=84, out_features=10, bias=True)
)

In [42]:
with torch.no_grad():   # 기울기계산 하지마
    total_corr = 0
    for img,labels in test_loader:
        img = img.to(device)
        labels = labels.to(device)

        preds = model(img)

        _, pred = torch.max(preds,1)     # 가장 큰 값, 예측된 값의 데이터를 집어넣고 1
        total_corr += ((pred==labels).sum().item())
        # print(pred)


    print(f'Acc {total_corr/len(test_data.targets)}')



Acc 0.9888
